In [ ]:
# Set-ExecutionPolicy Unrestricted -Scope Process
# .\.venv\Scripts\Activate    

Hello World


In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
1
NVIDIA GeForce RTX 4070 Laptop GPU


In [ ]:
import numpy as np
import json
import pandas as pd

# Load the LLaVA embeddings
embedding_data = np.load("./coco_dataset/llava_embeddings.npz", allow_pickle=True)
image_ids = embedding_data["image_ids"]
embeddings = embedding_data["embeddings"]

# Load captions
with open("./coco_dataset/llava_captions.json", "r") as f:
    captions = json.load(f)

# Prepare rows for DataFrame
rows = []
for img_id, emb in zip(image_ids, embeddings):
    padded_id = str(img_id).zfill(12)
    caption = captions.get(str(int(img_id)), "")  # some IDs may be unpadded in json keys
    row = {"image_id": padded_id, "caption": caption}
    row.update({f"dim_{i}": value for i, value in enumerate(emb)})
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)

# Save to CSV
csv_path = "./coco_dataset/llava_embeddings_with_captions.csv"
df.to_csv(csv_path, index=False)

: 

In [1]:
import ollama

In [3]:
print(ollama.list())

models=[Model(model='llava:latest', modified_at=datetime.datetime(2025, 3, 2, 14, 2, 7, 559507, tzinfo=TzInfo(+02:00)), digest='8dd30f6b0cb19f555f2c7a7ebda861449ea2cc76bf1f44e262931f45fc81d081', size=4733363377, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama', 'clip'], parameter_size='7B', quantization_level='Q4_0'))]


In [6]:
response = ollama.chat(
    model="llava",
    messages=[
        {"role": "user", "content": "What is in the image?", "images": ["./coco_dataset/train2017/000000000009.jpg"]}
    ],
)

print(response["message"]['role'])
print(response["message"]['content'])

assistant
 The image shows a plastic container filled with various food items, which appear to be part of a balanced meal. The container is divided into sections:

- On the left side, there are slices of fruit that could be apple and oranges.
- In the center, there is a sandwich that might contain some kind of meat or cheese, with what looks like a condiment on top, possibly mustard or mayonnaise.
- To the right, there's a serving of cooked vegetables that include broccoli and something that resembles chicken or pork.
- Below the vegetables, there appears to be a small container with a sauce or dip, such as ranch dressing or hummus, depending on the context.

The overall meal looks like it could be for lunch or dinner, offering a variety of food groups that include fruits, carbohydrates, proteins, and potentially some vegetables in the dip. 


In [ ]:
# import os
# import requests
# import zipfile

# coco_images_url = "http://images.cocodataset.org/zips/train2017.zip"
# coco_annotations_url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"

# dataset_path = "./coco_dataset"
# images_path = os.path.join(dataset_path, "train2017")
# annotations_path = os.path.join(dataset_path, "annotations")

# os.makedirs(images_path, exist_ok=True)
# os.makedirs(annotations_path, exist_ok=True)

# def download_and_extract(url, dest_folder):
#     zip_path = os.path.join(dest_folder, os.path.basename(url))
#     if not os.path.exists(zip_path):
#         print(f"Downloading {url} ...")
#         response = requests.get(url, stream=True)
#         with open(zip_path, "wb") as file:
#             for chunk in response.iter_content(chunk_size=1024):
#                 file.write(chunk)
    
#     print(f"Extracting {zip_path} ...")
#     with zipfile.ZipFile(zip_path, "r") as zip_ref:
#         zip_ref.extractall(dest_folder)
    
#     os.remove(zip_path)
#     print(f"Done extracting: {dest_folder}")

# download_and_extract(coco_images_url, dataset_path)
# download_and_extract(coco_annotations_url, dataset_path)


Extracting ./coco_dataset\train2017.zip ...
Done extracting: ./coco_dataset
Extracting ./coco_dataset\annotations_trainval2017.zip ...
Done extracting: ./coco_dataset


In [ ]:
# import os
# import random
# import shutil

# # Define paths
# dataset_path = "./coco_dataset"
# images_path = os.path.join(dataset_path, "train2017")  # Original COCO images
# train_path = os.path.join(dataset_path, "train")
# test_path = os.path.join(dataset_path, "test")

# # Create train and test directories
# os.makedirs(train_path, exist_ok=True)
# os.makedirs(test_path, exist_ok=True)

# # Get all image filenames
# all_images = [f for f in os.listdir(images_path) if f.endswith(".jpg")]

# # Shuffle images
# random.seed(42)  # For reproducibility
# random.shuffle(all_images)

# # Split: 80% Train, 20% Test
# split_index = int(0.8 * len(all_images))
# train_images = all_images[:split_index]
# test_images = all_images[split_index:]

# # Move images to respective folders
# for img in train_images:
#     shutil.move(os.path.join(images_path, img), os.path.join(train_path, img))

# for img in test_images:
#     shutil.move(os.path.join(images_path, img), os.path.join(test_path, img))

# print(f"Train set: {len(train_images)} images")
# print(f"Test set: {len(test_images)} images")


Train set: 94629 images
Test set: 23658 images


In [2]:
import json
import os

# Path to COCO captions file
annotations_path = "./coco_dataset/annotations/captions_train2017.json"

# Load the COCO captions JSON
with open(annotations_path, "r") as f:
    coco_data = json.load(f)

# Create a dictionary mapping image_id to captions
image_captions = {}
for annotation in coco_data["annotations"]:
    img_id = annotation["image_id"]
    caption = annotation["caption"]
    if img_id not in image_captions:
        image_captions[img_id] = []
    image_captions[img_id].append(caption)

print(f"Total images with captions: {len(image_captions)}")

Total images with captions: 118287


In [3]:
# Get the list of train/test image filenames (without extensions)
train_image_ids = {int(f.split(".")[0]) for f in os.listdir("./coco_dataset/train")}
test_image_ids = {int(f.split(".")[0]) for f in os.listdir("./coco_dataset/test")}

# Filter captions for train and test images
train_captions = {img_id: image_captions[img_id] for img_id in train_image_ids if img_id in image_captions}
test_captions = {img_id: image_captions[img_id] for img_id in test_image_ids if img_id in image_captions}

print(f"Train captions: {len(train_captions)} images")
print(f"Test captions: {len(test_captions)} images")


Train captions: 94629 images
Test captions: 23658 images


In [4]:
import ollama
import json
import os

# Path to save generated captions
llava_captions_path = "./coco_dataset/llava_captions.json"
checkpoint_interval = 100  # Save progress every 10,000 captions

def generate_description(image_path):
    """Generate a compact caption (max 20 words) using LLaVA."""
    response = ollama.chat(
        model="llava",
        messages=[
            {"role": "user", "content": "Describe this image briefly in 20 words or less.", "images": [image_path]}
        ],
    )
    description = response["message"]["content"]
    return " ".join(description.split())  # Ensure max 20 words


In [5]:
# Load existing captions if the script was interrupted before
if os.path.exists(llava_captions_path):
    with open(llava_captions_path, "r") as f:
        llava_captions = json.load(f)
else:
    llava_captions = {}


In [7]:
# Get all image paths (Train + Test)
image_paths = {
    img_id: f"./coco_dataset/train/{img_id:012d}.jpg" for img_id in train_captions.keys()
}
image_paths.update({
    img_id: f"./coco_dataset/test/{img_id:012d}.jpg" for img_id in test_captions.keys()
})

# Resume from last saved checkpoint
start_idx = len(llava_captions)

for i, (img_id, img_path) in enumerate(list(image_paths.items())[start_idx:], start=start_idx):
    if img_id in llava_captions:
        continue  # Skip already processed images

    try:
        llava_captions[img_id] = generate_description(img_path)
        print(f"Generated {i+1}/{len(image_paths)}: {llava_captions[img_id]}")

        # Save progress every 10,000 captions
        if (i + 1) % checkpoint_interval == 0:
            with open(llava_captions_path, "w") as f:
                json.dump(llava_captions, f, indent=4)
            print(f"✅ Checkpoint saved at {i+1} captions.")

    except Exception as e:
        print(f"❌ Error processing {img_id}: {e}")
        break  # Stop to avoid corrupt output

# Final save
with open(llava_captions_path, "w") as f:
    json.dump(llava_captions, f, indent=4)

print("✅ All captions generated and saved!")


Generated 112818/118287: The image shows two individuals seated at a dining table, each engrossed in their own laptop computer. One person is holding their chin thoughtfully while the other appears to be typing or browsing on their device. The table hosts several laptops and electronic devices. A smartphone is also visible on the table. The environment suggests a home setting with a comfortable atmosphere.
Generated 112819/118287: This is a composite image capturing a skateboarder's trick in several moments of time-lapse photography. The skater is captured in midair, performing a complex maneuver on a ramp. The background shows an urban setting with a building and a tree. There are two other people visible behind the action, possibly watching or waiting for their turn to skate.
Generated 112820/118287: The image depicts a vibrant street scene with a line of shops along one side. The shops are old and painted in bright colors, giving the area a lively and colorful atmosphere. In front o